<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import geopy
import requests
import folium

# from ipynb.fs.full.equal_groups import EqualGroupsKMeans

In [2]:
df = pd.read_csv('Data26_6.csv')

In [3]:
df.shape

(84, 45)

In [4]:
df_pickup = df.copy()
df_drop = df.copy()

In [5]:
df_pickup.drop(df_pickup[df_pickup.taskType == 'dropoff'].index, inplace=True)
df_drop.drop(df_drop[df_drop.taskType == 'pickup'].index, inplace=True)

In [6]:
df_pickup.shape

(17, 45)

In [7]:
df_drop.shape

(67, 45)

In [8]:
new_p = df_pickup["destinationLonLat"].str.split(",", n = 1, expand = True)
new_d = df_drop["destinationLonLat"].str.split(",", n = 1, expand = True)

In [9]:
new_p['name'] = df_pickup['destinationAddress']
new_d['name'] = df_drop['Pickup Address 1']+"_Customer"

In [10]:
new_p.isnull().sum()

0       0
1       0
name    0
dtype: int64

In [11]:
new_d.isnull().sum()

0       0
1       0
name    3
dtype: int64

In [12]:
new_d.dropna(inplace = True)
new_d.shape

(64, 3)

In [13]:
new_d.head()

,0,1,name
4,144.6937282,-37.8737748,24 Oceanwave Parade_Customer
5,144.6717971,-37.877901,24 Oceanwave Parade_Customer
6,144.6631264,-37.8340002,24 Oceanwave Parade_Customer
7,144.9662652,-37.8102205,24 Oceanwave Parade_Customer
8,145.0080457,-37.8818254,24 Oceanwave Parade_Customer


In [14]:
new_p.reset_index(drop=True, inplace=True)
new_d.reset_index(drop=True, inplace=True)
new_p.rename(columns={0: "lon", 1: "lat"}, inplace=True)
new_d.rename(columns={0: "lon", 1: "lat"}, inplace=True)

In [15]:
from geopy.geocoders import Nominatim 
# function to get the latitude and longitude for a locations
def get_lat_long(Address):
    geolocator = Nominatim(user_agent='a')
    location = geolocator.geocode(Address) 
    return(location.latitude, location.longitude)

In [16]:
# create map of Toronto using latitude and longitude values
import folium
latitude,longitude=get_lat_long('Victoria, VIC')
#print(latitude,longitude)
map_vic = folium.Map(location=[latitude, longitude], zoom_start=16)#,tiles='cartodb positron')
# add markers to map
for lat, lon, name in zip(new_p['lat'], new_p['lon'], new_p['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vic)  
    
map_vic

In [17]:
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt # plotting library

In [18]:
# Prepare the Data for the k-mean clustering for pickup
X=new_p[['lat','lon']]
#Prepare data for k-mean clustering of drop off
Y =new_d[['lat','lon']]

In [19]:
k_means_d = KMeans(init="k-means++", n_clusters=5, n_init=16)
k_means_d.fit(Y)
k_means_labels_d = k_means_d.labels_
k_means_cluster_centers_d = k_means_d.cluster_centers_
print ('clusters ] \n',k_means_labels_d ,'\n') ;
print('cluster centers \n' ,k_means_cluster_centers_d);

clusters ] 
 [1 1 1 0 4 3 3 4 0 3 4 4 0 4 0 4 4 0 4 3 4 4 4 4 3 3 4 0 4 0 0 4 0 2 2 4 0
 2 0 0 2 0 0 4 2 0 0 2 0 4 2 2 2 4 4 3 1 3 3 0 1 1 1 1] 

cluster centers 
 [[-37.79092042 144.98849622]
 [-37.86801628 144.69210666]
 [-37.93436758 145.2004213 ]
 [-37.77586196 144.8220704 ]
 [-37.87923146 145.08348702]]


In [20]:
new_d['Clusters'] = pd.DataFrame(k_means_labels_d)
new_d

,lon,lat,name,Clusters
0,144.6937282,-37.8737748,24 Oceanwave Parade_Customer,1
1,144.6717971,-37.877901,24 Oceanwave Parade_Customer,1
2,144.6631264,-37.8340002,24 Oceanwave Parade_Customer,1
3,144.9662652,-37.8102205,24 Oceanwave Parade_Customer,0
4,145.0080457,-37.8818254,24 Oceanwave Parade_Customer,4
...,...,...,...,...
59,144.9802227,-37.7690943,133 Chesterville Rd_Customer,0
60,144.7289906,-37.8251446,71 Queens Rd_Customer,1
61,144.666856,-37.8471829,357 Collins St_Customer,1
62,144.610518,-37.9036686,255 Camberwell Road_Customer,1


In [21]:
# Visualiza the created clusters drop off
import folium
latitude,longitude=get_lat_long('Victoria, VIC')
#print(latitude,longitude)
map_vic_clust_d = folium.Map(location=[latitude, longitude], zoom_start=16,tiles='cartodb positron')
cl=['#FF33EF','yellow','black','red','#33FF5D','#FFA833','#FF5733','#3AFF33','#FC33FF','#F9FF33','#3357FF']

for lat, lon, clust, name in zip(new_d['lat'], new_d['lon'],new_d['Clusters'], new_d['name']):
    label = '{},{}'.format(clust,name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color=cl[clust],
        fill_opacity=1,
        parse_html=False).add_to(map_vic_clust_d)  
    
map_vic_clust_d

In [22]:
new_d['Flag'] = 'false'
new_p['Clusters'] = '-1'
new_d

,lon,lat,name,Clusters,Flag
0,144.6937282,-37.8737748,24 Oceanwave Parade_Customer,1,false
1,144.6717971,-37.877901,24 Oceanwave Parade_Customer,1,false
2,144.6631264,-37.8340002,24 Oceanwave Parade_Customer,1,false
3,144.9662652,-37.8102205,24 Oceanwave Parade_Customer,0,false
4,145.0080457,-37.8818254,24 Oceanwave Parade_Customer,4,false
...,...,...,...,...,...
59,144.9802227,-37.7690943,133 Chesterville Rd_Customer,0,false
60,144.7289906,-37.8251446,71 Queens Rd_Customer,1,false
61,144.666856,-37.8471829,357 Collins St_Customer,1,false
62,144.610518,-37.9036686,255 Camberwell Road_Customer,1,false


In [23]:
new_p

,lon,lat,name,Clusters
0,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",-1
1,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",-1
2,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",-1
3,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",-1
4,144.9626893,-37.8170089,"357 Collins Street, Shop 2A, Melbourne, Victor...",-1
5,145.1905077,-37.91006,"36 Oak Terrace, Wheelers Hill, Victoria 3150",-1
6,145.0415827,-37.876045,"19 Derby Road, Caulfield East, Victoria 3145",-1
7,144.9627392,-37.8170843,"357 Collins Street, Melbourne, Victoria 3000",-1
8,144.9629631,-37.8155661,"353 Little Collins Street, Basement, Melbourne...",-1
9,145.3218496,-38.0744925,"112 Sabel Drive, Cranbourne North, Victoria 3977",-1


In [26]:
new_t1=[]
for i in new_p.index:
    for j in new_d.index:
        address = new_d['name'][j].split()[0] + " " + new_d['name'][j].split()[1]
        if address in new_p['name'][i] and new_d['Flag'][j] == 'false':
            new_p['Clusters'][i] = new_d['Clusters'][j]
            new_d['Flag'][j] = 'true'
            new_d.loc[(new_d.Clusters == new_d['Clusters'][j]) & (new_d.name == new_d['name'][j]), 'Flag'] = "true"
            break
# new_tt = pd.DataFrame(new_t1, columns=['Clusters'])
# new_test['Clusters'] = new_tt['Clusters']
# new_test.rename(columns={0: "lon", 1: "lat"}, inplace=True)
# new_test

new_p

<ipython-input-26-ce320bdb2eb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_d['Flag'][j] = 'true'
<ipython-input-26-ce320bdb2eb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_d['Flag'][j] = 'true'
<ipython-input-26-ce320bdb2eb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_d['Flag'][j] = 'true'
<ipython-input-26-ce320bdb2eb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,lon,lat,name,Clusters
0,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",1
1,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",0
2,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",4
3,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",3
4,144.9626893,-37.8170089,"357 Collins Street, Shop 2A, Melbourne, Victor...",0
5,145.1905077,-37.91006,"36 Oak Terrace, Wheelers Hill, Victoria 3150",1
6,145.0415827,-37.876045,"19 Derby Road, Caulfield East, Victoria 3145",4
7,144.9627392,-37.8170843,"357 Collins Street, Melbourne, Victoria 3000",2
8,144.9629631,-37.8155661,"353 Little Collins Street, Basement, Melbourne...",4
9,145.3218496,-38.0744925,"112 Sabel Drive, Cranbourne North, Victoria 3977",2


In [30]:
new_p.drop(new_p[new_p['Clusters'] == '-1'].index, inplace = True)
new_p

,lon,lat,name,Clusters
0,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",1
1,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",0
2,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",4
3,144.7406458,-37.9031938,"24 Oceanwave Parade, Point Cook, Victoria 3030",3
4,144.9626893,-37.8170089,"357 Collins Street, Shop 2A, Melbourne, Victor...",0
5,145.1905077,-37.91006,"36 Oak Terrace, Wheelers Hill, Victoria 3150",1
6,145.0415827,-37.876045,"19 Derby Road, Caulfield East, Victoria 3145",4
7,144.9627392,-37.8170843,"357 Collins Street, Melbourne, Victoria 3000",2
8,144.9629631,-37.8155661,"353 Little Collins Street, Basement, Melbourne...",4
9,145.3218496,-38.0744925,"112 Sabel Drive, Cranbourne North, Victoria 3977",2


In [33]:
# Update the Locations master dataframe to have all the restaurants and customer locations 
All_Locations=""
# All_Locations=pd.DataFrame([new_d.lon,new_d.lat,new_d.name+'_Customer',new_d.Clusters]).T
All_Locations=pd.DataFrame([new_d.lon,new_d.lat,new_d.name,new_d.Clusters]).T
All_Locations.head()
All_Locations=All_Locations.append(new_p,ignore_index=True)
print(All_Locations.shape)
All_Locations

            lon          lat  \
0   144.6937282  -37.8737748   
1   144.6717971   -37.877901   
2   144.6631264  -37.8340002   
3   144.9662652  -37.8102205   
4   145.0080457  -37.8818254   
..          ...          ...   
75  145.0839031  -37.9832959   
76  144.9626893  -37.8170089   
77  145.0578011  -37.8288172   
78   145.057637   -37.831846   
79  145.0585112  -37.9487927   

                                                 name Clusters  
0                        24 Oceanwave Parade_Customer        1  
1                        24 Oceanwave Parade_Customer        1  
2                        24 Oceanwave Parade_Customer        1  
3                        24 Oceanwave Parade_Customer        0  
4                        24 Oceanwave Parade_Customer        4  
..                                                ...      ...  
75  103 Lower Dandenong Road, Unit 2, Mentone, Vic...        4  
76  357 Collins Street, Shop 2A, Melbourne, Victor...        3  
77        656 Market Place, Ca

In [32]:
# Visualiza the created clusters drop off
import folium
latitude,longitude=get_lat_long('Victoria, VIC')
#print(latitude,longitude)
map_vic_clust = folium.Map(location=[latitude, longitude], zoom_start=16,tiles='cartodb positron')
cl=['#FF33EF','yellow','black','red','#33FF5D','#FFA833','#FF5733','#3AFF33','#FC33FF','#F9FF33','#3357FF']

for lat, lon, clust, name in zip(All_Locations['lat'], All_Locations['lon'],All_Locations['Clusters'], All_Locations['name']):
    label = '{},{}'.format(clust,name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color=cl[clust],
        fill_opacity=1,
        parse_html=False).add_to(map_vic_clust)  
    
map_vic_clust